In [9]:
import time
import numpy as np
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES']='-1'
import tensorflow as tf
import cv2

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from PIL import ImageGrab

mon = (0, 0, 750, 500)

In [10]:
# This is needed to display the images.
%matplotlib inline

In [11]:
from object_detection.utils import label_map_util
from object_detection.utils import ops as utils_ops
from object_detection.utils import visualization_utils as vis_util

In [12]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = 'frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'mscoco_label_map.pbtxt'

In [13]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [14]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [15]:
title = "FPS benchmark"
start_time = time.time()
display_time = 2 # displays the frame rate every 2 second
fps = 0
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        while True:
            image_np = np.array(ImageGrab.grab(bbox=mon))
            image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            #output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)

            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

            # Visualization of the results of a detection.
            (image_tensor , boxes , scores , classes , num_detections) = sess.run(
                [image_tensor , boxes , scores , classes , num_detections],
                feed_dict = {image_tensor:image_np_expanded})

            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=3)
            # Display output
            # Display the picture in grayscale
            cv2.imshow(title, cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY))   
            fps+=1
            TIME = time.time() - start_time
            if (TIME) >= display_time :
                print("FPS: ", fps / (TIME))
                fps = 0
                start_time = time.time()
            # Press "q" to quit
            if cv2.waitKey(25) & 0xFF == ord("q"):
                cv2.destroyAllWindows()
                break

FPS:  0.1430391271985386
FPS:  4.4331323559798825
FPS:  3.6015303677315447
FPS:  2.9938763463079985
FPS:  3.418920638883555
FPS:  3.201004779681122
FPS:  3.4908179167113023
FPS:  3.4054199759951413
FPS:  3.309597893499723
FPS:  4.192912418956064
FPS:  4.159722379097064
FPS:  3.883479072837963
FPS:  3.373940491349776
FPS:  3.6456599723227194
FPS:  4.333067103837079
